In [ ]:
from RawHandler.RawHandler import RawHandler
from RawHandler.utils import download_file_requests
import matplotlib.pyplot as plt
import os

In [ ]:
# Download example raw files
def download_file_if_needed(file_url, output_file):
    if not os.path.exists(output_file):
        download_file_requests(file_url, output_file)


file_url = "https://dataverse.uclouvain.be/api/access/datafile/:persistentId?persistentId=doi:10.14428/DVN/DEQCIM/UNYG7K"
output_file = "gtBark_GT.arw"
download_file_if_needed(file_url, output_file)

In [ ]:
# Load the first image
rh = RawHandler("gtBark_GT.arw", colorspace="sRGB")

In [ ]:
# We can acccess the underlying bayer data
# dims = (h1, h2, w1, w2)
dims = (1500, 1500 + 200, 4500, 4500 + 200)
bayer = rh.raw
plt.imshow(bayer[dims[0] : dims[1], dims[2] : dims[3]])

In [ ]:
# Or, we can adjust the black and white point, apply a color space transform, and crop using the apply_colorspace_transform.
bayer = rh.apply_colorspace_transform(dims=dims)
plt.imshow(bayer.transpose(1, 2, 0))
bayer.min(), bayer.max()

In [ ]:
# However, chances are we want two representations: the rggb resentation of the bayer array and a demosaiced ground truth.
# The rggb representation stacks the 4 pixels of the bayer pattern into one. Naturally, this has 1/2 the linear size of the original image.
rggb = rh.as_rggb(dims=dims)
plt.subplots(1, 4, figsize=(16, 4))
for idx, channel in enumerate("RGGB"):
    plt.subplot(1, 4, idx + 1)
    plt.imshow(rggb[idx])
    plt.title(channel)

In [ ]:
# To draw an demosaiced rgb image, we can use as_rgb, which returns an image with dimensions C, H, W. We also can apply a gamma curve.
from RawHandler.utils import linear_to_srgb

rgb = rh.as_rgb(dims=dims)
plt.imshow(linear_to_srgb(rgb).transpose(1, 2, 0))
rgb.min(), rgb.max()

In [ ]:
# Alternate colorspaces can be applied
from RawHandler.utils import linear_to_srgb

rgb = rh.as_rgb(dims=dims, colorspace="XYZ")
plt.imshow(linear_to_srgb(rgb).transpose(1, 2, 0))
rgb.min(), rgb.max()

In [ ]:
# We can also generate a sparse representation
from RawHandler.utils import linear_to_srgb

rgb = rh.as_sparse(dims=dims, colorspace="XYZ")
plt.imshow(linear_to_srgb(rgb).transpose(1, 2, 0))
rgb.min(), rgb.max()

In [ ]:
## We can also generate a small preview of the whole image. Clip=True can be used to clip the final image to 0-1.

img = rh.generate_thumbnail(clip=True)
plt.imshow(linear_to_srgb(img))